In [8]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Carpeta con las imágenes
img_folder = 'img_align_celeba/img_align_celeba'
img_names = os.listdir(img_folder)

def preprocess_image(image_path, target_size=(64, 64)):
    # Cargar y redimensionar la imagen
    original_image = load_img(image_path, target_size=target_size)

    # Convertir la imagen al formato Numpy
    numpy_image = img_to_array(original_image)

    # Normalizar la imagen al rango [-1, 1]
    numpy_image = (numpy_image - 127.5) / 127.5
    return numpy_image



In [9]:
def load_batch(img_names, batch_size, img_folder, img_size=(64, 64)):
    batch_start = 0
    batch_end = batch_size

    while batch_start < len(img_names):
        limit = min(batch_end, len(img_names))
        img_batch = [preprocess_image(os.path.join(img_folder, img_names[i]), target_size=img_size) for i in range(batch_start, limit)]
        yield np.array(img_batch)

        batch_start += batch_size
        batch_end += batch_size


In [10]:
from keras.layers import Dense, Reshape, LeakyReLU, BatchNormalization, UpSampling2D, Conv2D
from keras.models import Sequential

def create_generator():
    model = Sequential()

    model.add(Dense(256 * 8 * 8, input_dim=100))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((8, 8, 256)))

    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(0.2))

    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(0.2))

    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(3, kernel_size=3, padding='same', activation='tanh'))
    return model


In [11]:
from keras.layers import Flatten, Dropout

def create_discriminator():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(64, 64, 3), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model


In [12]:
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input

def create_gan(generator, discriminator):
    optimizer = Adam(0.0002, 0.5)
    
    discriminator.trainable = False

    z = Input(shape=(100,))
    img = generator(z)
    validity = discriminator(img)

    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    return combined


In [13]:
generator = create_generator()
discriminator = create_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
combined = create_gan(generator, discriminator)



In [14]:
import matplotlib.pyplot as plt


def train_gan(generator, discriminator, combined, img_names, img_folder, epochs, batch_size, save_interval):
    
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))


    for epoch in range(epochs):

        batch_index = 0

        for imgs in load_batch(img_names, batch_size, img_folder):

            # ---------------------
            #  Entrenar el Discriminador
            # ---------------------
            noise = np.random.normal(0, 1, (batch_size, 100))
            generated_imgs = generator.predict(noise)

            d_loss_real = discriminator.train_on_batch(imgs, valid)
            d_loss_fake = discriminator.train_on_batch(generated_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Entrenar el Generador
            # ---------------------
            g_loss = combined.train_on_batch(noise, valid)

            print(batch_index)
            batch_index +=1
            if batch_index % 50 == 0:
                save_generated_imgs(epoch)


        print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]}] [G loss: {g_loss}]")


def save_generated_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    generated_imgs = generator.predict(noise)
    generated_imgs = 0.5 * generated_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(generated_imgs[cnt, :,:,:])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(f"./images/generated_{epoch}.png")
    plt.close()

# Llamar a la función para entrenar
train_gan(generator, discriminator, combined, img_names, img_folder, epochs=5, batch_size=250, save_interval=1)


8/8 [==============================] - 1s 139ms/step
0
8/8 [==============================] - 1s 113ms/step
1
8/8 [==============================] - 1s 119ms/step
2
8/8 [==============================] - 1s 124ms/step
3
8/8 [==============================] - 1s 118ms/step
4
8/8 [==============================] - 1s 119ms/step
5
8/8 [==============================] - 1s 123ms/step
6
8/8 [==============================] - 1s 116ms/step
7
8/8 [==============================] - 1s 115ms/step
8
8/8 [==============================] - 1s 114ms/step
9
8/8 [==============================] - 1s 116ms/step
10
8/8 [==============================] - 1s 114ms/step
11
8/8 [==============================] - 1s 126ms/step
12
8/8 [==============================] - 1s 116ms/step
13
8/8 [==============================] - 1s 118ms/step
14
8/8 [==============================] - 1s 118ms/step
15
8/8 [==============================] - 1s 118ms/step
16
8/8 [==============================] - 1s 116ms/step
17
8/